In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import gc

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [ ]:
%matplotlib inline

In [ ]:
path = r'C:\Users\nukis\Documents\Projects\08. Road Safety'

In [ ]:
df_crash = pd.read_pickle(os.path.join(path, '01. Data', 'Prepared data', 'crash_cleaned.pkl'))

In [ ]:
df_uncas = pd.read_pickle(os.path.join(path, '01. Data', 'Prepared data', 'unitcasualty_cleaned.pkl'))

### 2. Merging Dataframes

In [ ]:
df = df_crash.merge(df_uncas, on = 'REPORT_ID', how = 'left')

In [ ]:
# Command to maximize view of rows and columns

pd.options.display.max_rows = None
pd.options.display.max_columns = len(df.columns)

In [ ]:
df.shape

In [ ]:
df.info()

### 3. Data Cleaning

In [ ]:
# Cleaning Space

cat = df.select_dtypes(include = ['object']).columns

for i in cat:
    try:
        df[i] = df[i].apply(lambda x: x.strip())
        df[i] = df[i].apply(lambda x: x.title())
        df.loc[(df[i] == 'Unknown') | (df[i] == 'N/A') | (df[i] == 'Xx') | (df[i] == 'Xxx') | (df[i] == 'Xxxx'), i] = np.NaN
    except:
        print(i)

In [ ]:
# Check for missing values

df.isnull().sum()

In [ ]:
# Check for missing values

pd.DataFrame(data = [round(i/len(df) * 100, 2) for i in df.isnull().sum().to_list()], index = df.columns, columns = ['Missing Values %']).T

In [ ]:
# Dropping the missing values from data

df.dropna(inplace=True)
df.shape

In [ ]:
# Check again for missing values

df.isnull().sum()

In [ ]:
# Check for duplicates

dups = df.duplicated(keep = 'last')
dups.sum() 

In [ ]:
# Drop duplicates

df = df.drop_duplicates()
dups = df.duplicated()
dups.sum()

In [ ]:
# Check for mixed-type data in dataframe

for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col) # No mixed-type

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Convert data type

df['Age'] = df['Age'].astype('int64')
df['DUI Involved'] = df['DUI Involved'].astype('int64')
df['Drugs Involved'] = df['Drugs Involved'].astype('int64')
df['Lat'] = df['Lat'].astype('float64')
df['Lon'] = df['Lon'].astype('float64')
df['Veh Year'] = df['Veh Year'].astype('float64')

### 4. Data Pre-Prossesing

#### Hour Grouping
##### Hour is converted into hourly basis.

In [ ]:
hourly = []

for i in df['Hour']:
    n = 2
    i = i[:n]
        
    hourly.append(i)

In [ ]:
df['Hourly'] = hourly
df['Hourly'] = df['Hourly'].astype('int64')

In [ ]:
df = df.drop(columns = ['REPORT_ID', 'Hour'])

#### Day Grouping

In [ ]:
df.loc[(df['Day'] == 'Monday') | (df['Day'] == 'Tuesday') | (df['Day'] == 'Wednesday') |
           (df['Day'] == 'Thursday') | (df['Day'] == 'Friday'), 'Day Group'] = 'Weekday'
df.loc[(df['Day'] == 'Saturday') | (df['Day'] == 'Sunday'), 'Day Group'] = 'Weekend'

#### Month Grouping

In [ ]:
df.loc[(df['Month'] == 'January') | (df['Month'] == 'February') | (df['Month'] == 'March'), 'Month Group'] = 'Q1'
df.loc[(df['Month'] == 'April') | (df['Month'] == 'May') | (df['Month'] == 'June'), 'Month Group'] = 'Q2'
df.loc[(df['Month'] == 'July') | (df['Month'] == 'August') | (df['Month'] == 'September'), 'Month Group'] = 'Q3'
df.loc[(df['Month'] == 'October') | (df['Month'] == 'November') | (df['Month'] == 'December'), 'Month Group'] = 'Q1'

#### Target Column

In [ ]:
# Changing name of the target column

df = df.rename(columns = {'CSEF Severity' : 'Target'})

In [ ]:
target_trim = []

for i in df['Target']:
    target_trim.append(i[3:])

In [ ]:
df['Target'] = target_trim

In [ ]:
df.info()

In [ ]:
# Export data to pkl

df.to_pickle(os.path.join(path, '01. Data', 'Prepared data', 'road_safety_cleaned.pkl'))
df.to_csv(os.path.join(path, '01. Data', 'Prepared data', 'road_safety_cleaned.csv'), sep = ',')

#### Accident Severity Analysis

In [ ]:
labels = ['Property Damage Only', 'Minor Injury', 'Serious Injury', 'Fatality']
colors = ['#FEF9A7', '#FAC213', '#F77E21', '#D61C4E']

fig, ax = plt.subplots()
myexplode = (0.05, 0.05, 0.05, 0.2)

ax.pie(df['Target'].value_counts(), explode = myexplode, labels = labels,autopct='%1.1f%%', 
        wedgeprops={'linewidth': 3.0, 'edgecolor': 'white'}, startangle = 90, colors = colors)

ax.set_title('Accident Severity', fontsize = 14)
plt.show()

In [ ]:
# Function for drawing countplot

def countplot(x):
    plt.figure(figsize = (8, 6))
    sns.countplot(data = df, x = x, palette = 'mako_r', 
                  order = df[x].value_counts().index)

In [ ]:
countplot('Target')

#### As we can see, there are 4 classes of severity. We can see that the distribution of the classes is greatly disbalanced. 'Property Damage Only (PDO)' class is in majority while 'Fatality (Fatal)' class is the minority here.
#### Due to imbalance dataset, binary classification will be performed. For this purpose, classes of Mi, Si and Fatal will be grouped together as Injury/Death.

In [ ]:
# Grouping Minor injury, Serious injury abd fatality into one class

df.loc[df['Target'] == 'Mi', 'Target'] = 'Injury/Death'
df.loc[df['Target'] == 'Si', 'Target'] = 'Injury/Death'
df.loc[df['Target'] == 'Fatal', 'Target'] = 'Injury/Death'
df.loc[df['Target'] == 'Pdo', 'Target'] = 'PDO'

In [ ]:
labels = ['PDO', 'Injury/Death']
colors = ['#FEF9A7', '#D61C4E']

fig, ax = plt.subplots()
myexplode = (0.05, 0.05)

ax.pie(df['Target'].value_counts(), explode = myexplode, labels = labels,autopct='%1.1f%%', 
        wedgeprops={'linewidth': 3.0, 'edgecolor': 'white'}, startangle = 90, colors = colors)

ax.set_title('Accident Severity', fontsize = 14)
plt.show()

In [ ]:
countplot('Target')

In [ ]:
df.describe()

In [ ]:
# There is outliers in Vehicle Year column: 1900 will be removed

df = df.loc[df['Veh Year'] >= 1940]

In [ ]:
df.shape

#### Label-Encoding for Target Column

In [ ]:
labelencoder = LabelEncoder()

In [ ]:
df['Target'] = labelencoder.fit_transform(df['Target']) # Target column
df['Target'] = df['Target'].astype('int64')

dict(zip(labelencoder.inverse_transform([0,1]),[0,1]))

### 5. One-Hot-Encoding for categorical data
#### As many of the features are categorical, One-Hot-Encoding is performed.

In [ ]:
# Creating categorical features list

catvar = df.select_dtypes(include = ['object']).columns
catvar

In [ ]:
# Creating one hot encoder object 

onehotencoder = OneHotEncoder(handle_unknown = 'ignore') # Whether to raise an error

In [ ]:
# Fit and transform the data using the .fit_transform() method
# return the array version of the transformed data using the .toarray() method

df_enc = onehotencoder.fit_transform(df[catvar]).toarray()
df_enc

In [ ]:
feature_array = onehotencoder.get_feature_names_out()
feature_array

In [ ]:
# Convert to dataframe

df_enc = pd.DataFrame(df_enc, columns = feature_array)
df_enc.head()

In [ ]:
df_enc.shape

In [ ]:
# Concatenate with the dataframe

df_num = df.drop(columns = catvar, axis = 1).reset_index()
df_num = df_num.drop(columns = 'index', axis = 1)
df_num.shape

In [ ]:
df_new = pd.concat([df_num, df_enc], axis=1)
df_new.shape

In [ ]:
# Export data to pkl

df_new.to_pickle(os.path.join(path, '01. Data', 'Prepared data', 'road_safety_encoded.pkl'))
df_new.to_csv(os.path.join(path, '01. Data', 'Prepared data', 'road_safety_encoded.csv'), sep = ',')

### 6. Feature Selection
#### A. Filter Method (Pearson Correlation)

In [ ]:
X = df_new.drop(columns = 'Target') # Features
y = df_new['Target'] # Dependent variable (Target)

In [ ]:
cor = df_new.corr() # It can be plotted using heatmap, but there are too many features

In [ ]:
cor_target = abs(cor['Target'])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > 0.5]
relevant_features

##### Here only Total Causalty is highly correlated with the target variable. From data perspective, it makes sense that casualty number correlates to the crash severity.
##### However, from road safety perspective, some other factors shoud also play a role, such as road condition, driver conditiion, etc. Therefore, this feature selection method will not be considered.
##### In other words, this method is less accurate.

#### B. Wrapper Method (RFE (Recursive Feature Elimination))

##### It works by recursively removing attributes and building a model on those attributes that remain. It uses accuracy metric to rank the feature according to their importance. 
##### The RFE method takes the model to be used and the number of required features as input. It then gives the ranking of all the variables, 1 being most important. It also gives its support, True being relevant feature and False being irrelevant feature.

In [ ]:
model = LinearRegression()

In [ ]:
rfe = RFE(model, n_features_to_select = 15) #15 is random selection

In [ ]:
#Transforming data using RFE

X_rfe = rfe.fit_transform(X,y) 

In [ ]:
#Fitting the data to model
model.fit(X_rfe,y)

print(rfe.support_)
print(rfe.ranking_)

In [ ]:
#no of features

nof_list = np.arange(1, 15)            
high_score = 0

#Variable to store the optimum features

nof = 0           
score_list = []

for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    
    model = LinearRegression()
    rfe = RFE(model, n_features_to_select=nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    
    model.fit(X_train_rfe,y_train)
    
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    
    if(score>high_score):
        high_score = score
        nof = nof_list[n]

print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

##### As seen from above code, the optimum number of features is 12.
##### We now feed 12 as number of features to RFE and get the final set of features given by RFE method.

In [ ]:
cols = list(X.columns)
model = LinearRegression() #Initializing RFE model

rfe = RFE(model, n_features_to_select = 12) #Transforming data using RFE

X_rfe = rfe.fit_transform(X,y)  #Fitting the data to model
model.fit(X_rfe,y)

temp = pd.Series(rfe.support_, index = cols)
selected_features_rfe = temp[temp == True].index
print(selected_features_rfe)

#### From this method, it seems that place and time features play a role, which might be a hint to include place- and time-related features into the model.
#### For sake of validation, embedded method is performed as follows.

#### C. Embedded Method

In [ ]:
reg = LassoCV()
reg.fit(X, y)

print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
imp_coef

In [ ]:
selected_features = imp_coef.loc[(imp_coef > 1e-2) | (imp_coef < -0)]
selected_features 

In [ ]:
plt.rcParams['figure.figsize'] = (20, 10)
selected_features.plot(kind = 'barh')
plt.title('Feature importance using Lasso Model')

##### Embeded method seems also to give the same tendency. Therefore, all place- and time-related features will be selected.

#### D. Selecting Features

In [ ]:
features = ['Target', 'Total Cas', 'Total Units', 'Total SI', 'Total MI', 'Total Fats', 
            'DUI Involved', 'Drugs Involved', 'DayNight_Daylight', 'DayNight_Night', 
            'Month_April', 'Month_August','Month_December', 'Month_February', 'Month_January', 'Month_July',
       'Month_June', 'Month_March', 'Month_May', 'Month_November',
       'Month_October', 'Month_September', 'Day_Friday', 'Day_Monday',
       'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday',
       'Day_Wednesday', 'Crash Type_Hit Animal', 'Crash Type_Hit Fixed Object',
       'Crash Type_Hit Object On Road', 'Crash Type_Hit Parked Vehicle',
       'Crash Type_Hit Pedestrian',
       'Crash Type_Left Road - Out Of Control', 'Crash Type_Other',
       'Crash Type_Rear End', 'Crash Type_Right Angle',
       'Crash Type_Right Turn', 'Crash Type_Roll Over',
       'Crash Type_Side Swipe', 'Direction Of Travel_North', 'Direction Of Travel_North East',
       'Direction Of Travel_North West', 'Direction Of Travel_South',
       'Direction Of Travel_South East', 'Direction Of Travel_South West',
       'Direction Of Travel_West', 'Traffic Ctrls_Give Way Sign', 'Traffic Ctrls_No Control',
       'Traffic Ctrls_Other', 'Traffic Ctrls_Rail Xing - Boom',
       'Traffic Ctrls_Rail Xing - Flashing',
       'Traffic Ctrls_Rail Xing - No Control',
       'Traffic Ctrls_Rail Xing-Traffic Signals', 'Day Group_Weekday', 'Day Group_Weekend', 'Position Type_Crossover', 'Position Type_Divided Road',
       'Position Type_Freeway', 'Position Type_Interchange',
       'Position Type_Multiple', 'Position Type_Not Divided',
       'Position Type_One Way', 'Position Type_Other',
       'Position Type_Pedestrian Crossing', 'Position Type_Rail Crossing',
       'Position Type_Rail Xing', 'Position Type_Ramp Off',
       'Position Type_Ramp On', 'Position Type_T-Junction',
       'Position Type_Y-Junction', 'Sex_Female', 'Sex_Male', 'Age', 'Vertical Align_Crest Of Hill', 'Vertical Align_Level',
       'Vertical Align_Slope', 'Weather Cond_Not Raining', 'Weather Cond_Raining']

In [ ]:
df_new2 = df_new[features]
df_new2.shape

### 6. Building model

#### A Decision Tree is a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules (if-else) inferred from the data features.

#### Preparing and Splitting Dataset

In [ ]:
X = df_new2.drop(columns = 'Target') # Features
y = df_new2['Target'] # Dependent variable (Target)

In [ ]:
y.value_counts()

In [ ]:
# Divide into training-set and test-set: 80% 20% ratio

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#### Logistic Regression

In [ ]:
lgr = LogisticRegression()

In [ ]:
lgr.fit(X_train, y_train)

In [ ]:
y_pred = lgr.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))

In [ ]:
pd.DataFrame(lgr.coef_, columns=X_test.columns, index=['n']).T.sort_values(by='n', key=abs)

#### Decision Tree Classifier Model - Entropy

In [ ]:
# Create a Decision Tree object.

dtc = DecisionTreeClassifier(criterion = 'entropy')

In [ ]:
# Fit the DTC object onto the training set.

dtc.fit(X_train, y_train)

In [ ]:
y_pred = dtc.predict(X_test)

In [ ]:
print('Train Accuracy:', accuracy_score(y_train, dtc.predict(X_train)))

In [ ]:
print('Test Accuracy:', '{:.2f}'.format(accuracy_score(y_test, y_pred)))

In [ ]:
TN, FP, FN, TP = confusion_matrix(y_test, y_pred).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)


In [ ]:
pd.Series(y_pred).value_counts()

In [ ]:
pd.Series(y_test).value_counts()

In [ ]:
feature_cols = []

for i in df_new2:
    if i != 'Target':
        feature_cols.append(i)

In [ ]:
# Visualising Decision Tree

plt.figure(figsize = (25, 10))

dec_tree = plot_tree(dtc, feature_names = feature_cols, class_names = ['0', '1'],
                    filled = True, rounded = True, fontsize = 14)
plt.savefig('decisiontree.png')

In [ ]:
# Create a RF object.

rfc = RandomForestClassifier()

In [ ]:
# Fit the DTC object onto the training set.

rfc.fit(X_train, y_train)

In [ ]:
y_pred = rfc.predict(X_test)

In [ ]:
print('Train Accuracy:', accuracy_score(y_train, rfc.predict(X_train)))

In [ ]:
print('Test Accuracy:', '{:.2f}'.format(accuracy_score(y_test, y_pred)))

In [ ]:
gc.collect()